In [1]:
import numpy as np
import tensorflow as tf

from recommender import Recommender

import pandas as pd

In [2]:


test_list_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test_lists_ds").cache()
portfolios = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/portfolios_tfds").cache()

items_ids = portfolios.batch(10000).map(lambda x: x["STOCKCODE"])
unique_item_ids = np.unique(np.concatenate(list(items_ids)))

item_names = portfolios.batch(10000).map(lambda x: x["STOCKNAME"])
unique_item_names = np.unique(np.concatenate(list(item_names)))

item_GICS = portfolios.batch(10000).map(lambda x: x["GICS"])
unique_item_gics = np.unique(np.concatenate(list(item_GICS)))

user_ids = portfolios.batch(10000).map(lambda x: x["CDSACCNO"])
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

model = Recommender(
    loss = tf.keras.losses.MeanSquaredError(),
    portfolios = portfolios
    )

model.load_weights(r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_04_25\tf_listwise_ranking_2024_04_25_12_49")

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.evaluate(test_list_ds.shuffle(100_000).batch(1024), return_dict=True)

(None, 5, 32)  |  (None, 5, 64)
(None, 5, 32)  |  (None, 5, 64)
1/1 [==============================] - 3s 3s/step - ndcg_metric: 0.8401 - root_mean_squared_error: 1.2172 - loss: 1.4817 - regularization_loss: 0.0000e+00 - total_loss: 1.4817


{'ndcg_metric': 0.8400689363479614,
 'root_mean_squared_error': 1.2172322273254395,
 'loss': 1.481654167175293,
 'regularization_loss': 0,
 'total_loss': 1.481654167175293}

In [4]:
symbol_uni_path = r"D:\dev work\recommender systems\Atrad_CARS\data\stock_data.xlsx"
stock_info = pd.read_excel(symbol_uni_path)
stock_info = stock_info.drop(['Unnamed: 0'],axis = 1)
stock_info.shape


(282, 4)

In [5]:
stock_info.head(3)

,symbol,name,buisnesssummary,gics_code
0,HBS,hSenid Business Solutions PLC,An indigenous multinational catering towards m...,45103010 - Application Software
1,TYRE,KELANI TYRES PLC,Manufacturing tyres and tubes and marketing lo...,Automobiles & Components
2,ABL,AMANA BANK PLC,unknown,Banks


In [6]:
len(stock_info['symbol'].to_list())


282

In [7]:
symbol_universe = np.array(stock_info['symbol'].to_list())
name_universe = np.array(stock_info['name'].to_list())
gics_universe = np.array(stock_info['gics_code'].to_list())


In [8]:
test_1 = next(iter(test_list_ds.batch(2)))
test_1

{'GICS': <tf.Tensor: shape=(2, 5), dtype=string, numpy=
 array([[b'Capital Goods', b'FOOD BEVERAGE & TOBACCO',
         b'FOOD BEVERAGE & TOBACCO', b'Retailing',
         b'Diversified Financials'],
        [b'Retailing', b'Energy', b'Commercial & Professional Services',
         b'Capital Goods', b'Food Beverage & Tobacco']], dtype=object)>,
 'STOCKNAME': <tf.Tensor: shape=(2, 5), dtype=string, numpy=
 array([[b'KELANI CABLES PLC', b'BAIRAHA FARMS PLC',
         b'CEYLON COLD STORES PLC', b'R I L PROPERTY PLC',
         b'MERCHANT BANK OF SRI LANKA & FINANCE PLC'],
        [b'DIESEL & MOTOR ENGINEERING PLC', b'LANKA IOC PLC',
         b'E M L CONSULTANTS PLC', b'HAYLEYS PLC',
         b'KAHAWATTE  PLANTATIONS  PLC']], dtype=object)>,
 'UNIX_TS': <tf.Tensor: shape=(2, 5), dtype=float32, numpy=
 array([[1.6430490e+09, 1.6431354e+09, 1.6625754e+09, 1.6895322e+09,
         1.6441722e+09],
        [1.6636986e+09, 1.6649082e+09, 1.6589466e+09, 1.6612794e+09,
         1.6623162e+09]], dtype=

In [9]:
test_1['CDSACCNO'].numpy()

array([b'RPS-20882-LI/00', b'RPS-31648-LI/00'], dtype=object)

In [26]:
user = {
    'CDSACCNO' : test_1['CDSACCNO'].numpy(),
    'STOCKCODE' : symbol_universe[:10].reshape(-1,5),
    'GICS' : gics_universe[:10].reshape(-1,5),
    'STOCKNAME' : name_universe[:10].reshape(-1,5)
    }

In [27]:
symbol_universe[:10].reshape(-1,5)

array([['HBS', 'TYRE', 'ABL', 'DFCC', 'COMB'],
       ['HNB', 'HDFC', 'NTB', 'NDB', 'PABC']], dtype='<U4')

In [28]:
user

{'CDSACCNO': array([b'RPS-20882-LI/00', b'RPS-31648-LI/00'], dtype=object),
 'STOCKCODE': array([['HBS', 'TYRE', 'ABL', 'DFCC', 'COMB'],
        ['HNB', 'HDFC', 'NTB', 'NDB', 'PABC']], dtype='<U4'),
 'GICS': array([['45103010 - Application Software', 'Automobiles & Components',
         'Banks', 'Banks', 'Banks'],
        ['Banks', 'Banks', 'Banks', 'Banks', 'Banks']], dtype='<U34'),
 'STOCKNAME': array([['hSenid Business Solutions PLC', 'KELANI TYRES PLC',
         'AMANA BANK PLC', 'DFCC BANK PLC',
         'COMMERCIAL BANK OF CEYLON PLC'],
        ['HATTON NATIONAL BANK PLC',
         'HOUSING DEVELOPMENT FINANCE CORPORATION BANK OF SL',
         'NATIONS TRUST BANK PLC', 'NATIONAL DEVELOPMENT BANK PLC',
         'PAN ASIA BANKING CORPORATION PLC']], dtype='<U50')}

In [38]:
model(user)

(2, 5, 32)  |  (2, 5, 64)


<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[1.4720235],
        [1.931324 ],
        [2.502866 ],
        [2.4485707],
        [2.7979016]],

       [[1.4556653],
        [1.3368728],
        [1.8625946],
        [1.7156005],
        [1.2535155]]], dtype=float32)>